In [1]:
!pip install pandas scikit-learn pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 10.1 MB/s  0:00:01 10.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 9.0 MB/s  0:00:009.1 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 9.9 MB/s  0:00:03m 9.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 11.2 MB/s  0:00:001.8 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 10.7 MB/s  0:00:010.8 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [scikit-learn]0m 7/8 [scikit-learn]


In [2]:
import pandas as pd

# Yellow taxi Jan 2023
jan_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
df_jan = pd.read_parquet(jan_url)

df_jan.shape


(3066766, 19)

In [3]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

df_jan['duration'].std()


np.float64(42.59435124195457)

In [4]:
df_clean = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]

len(df_clean) / len(df_jan)


0.9812202822125979

In [5]:
from sklearn.feature_extraction import DictVectorizer

train_dicts = df_clean[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

X_train.shape


(3009173, 515)

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

y_train = df_clean['duration'].values

model = LinearRegression()
model.fit(X_train, y_train)

preds = model.predict(X_train)

rmse_train = np.sqrt(mean_squared_error(y_train, preds))
rmse_train


np.float64(7.6492619593358615)

In [7]:
feb_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
df_feb = pd.read_parquet(feb_url)

df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60
df_feb_clean = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]


In [8]:
val_dicts = df_feb_clean[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_feb_clean['duration'].values


In [9]:
pred_val = model.predict(X_val)

rmse_val = np.sqrt(mean_squared_error(y_val, pred_val))
rmse_val


np.float64(7.811816898533222)